In [18]:
import numpy as np
import pygame as pg
import math
import time
from Agents.agent import Agent
from Agents.cast_update_rays import update_cast_rays

In [3]:
WIDTH = 600
HEIGHT = 600
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

In [64]:
class Turret:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.pos_x = width / 2
        self.pos_y = height / 2
        self.position = np.array([self.pos_x, self.pos_y], dtype=np.float32)
        # self.screen = screen
        self.angle = 0
        self.radius = 15

        self.shoot_interval = 2  # shooting interval of the turret
        self.center = (int(self.pos_x), int(self.pos_y))

        # self.rect = pg.draw.circle(screen, GREEN, self.center, self.radius)

        self.bullet_pos = (self.center[0] + self.radius * math.cos(math.radians(self.angle)),
                           self.center[1] + self.radius * math.sin(math.radians(self.angle)))

        self.bullets = []

    def rotate_turret(self, end_point):
        """
        the end_point here gives a position coordinate (x, y)
        this needs to be used for the turret direction
        """

        magnitude = 30

        direction = end_point - self.position
        angle = math.atan2(direction[1], direction[0])
        angle = math.degrees(angle)
        angle = (angle + 360) % 360
        self.angle = angle
        angle = math.radians(angle)

        dir_vec_x = magnitude * math.cos(angle)
        dir_vec_y = magnitude * math.sin(angle)

        direction_end = (self.center[0] + dir_vec_x, self.center[1] + dir_vec_y)

        self.bullet_pos = (self.center[0] + self.radius * math.cos(angle), self.center[1] + self.radius * math.sin(angle))
        # self.bullet_pos = direction_end / 2

        return direction_end

    def shoot(self):
        bullet = Bullet(self.bullet_pos[0], self.bullet_pos[1], self.angle)
        self.bullets.append(bullet)

    def get_bullets(self):
        return self.bullets
    
    def auto_destroy(self):
        if self.bullets[0].pos[0] + 30 > self.width or self.bullets[0].pos[1] + 30 > self.height or \
            self.bullets[0].pos[0] - 30 < 0 or self.bullets[0].pos[1] - 30 < 0:
            self.destroy_bullet(self.bullets[0])
            
    def destroy_bullet(self, bullet):
        self.bullets.remove(bullet)


In [37]:
direction = np.array([70, 50]) - np.array([100, 50])
angle = math.atan2(direction[1], direction[0])
angle = math.degrees(angle)
angle = (angle + 360) % 360
angle

180.0

In [56]:
class Bullet:
    def __init__(self, x, y, angle):
        self.width = 2
        self.height = 5
        self.angle = angle
        self.speed = 5
        # self.screen = screen
        self.pos_x = x
        self.pos_y = y
        self.pos = np.array([self.pos_x, self.pos_y], dtype=np.float32)
        self.center = (int(self.pos_x), int(self.pos_y))
        self.radius = 10

        # this rect object is for drawing
        # self.rect = pg.draw.circle(self.screen, RED, self.center, self.radius)

    def move(self):
        angle = math.radians(self.angle)
        # self.pos_x += self.speed * math.cos(angle)
        # self.pos_y += self.speed * math.sin(angle)
        # angle = self.angle
        dir_end_x = self.pos[0] + self.speed * math.cos(angle)
        dir_end_y = self.pos[1] + self.speed * math.sin(angle)

        dir_end = np.array([dir_end_x, dir_end_y], dtype=np.float32)
        direction = dir_end - self.pos
        direction /= np.linalg.norm(direction)

        # draw_dir_end = (int(dir_end_x), int(dir_end_y))

        # self.angle = self.angle % 360

        self.pos = self.pos + direction * self.speed
        # pos = (self.pos[0], self.pos[1])
        self.center = (int(self.pos[0]), int(self.pos[1]))

        # self.rect = pg.draw.circle(self.screen, RED, self.center, self.radius)


In [5]:
angle = math.radians(100)
x=300 + 30 * math.cos(angle)
y=300 + 30 * math.sin(angle)
print(x, y)

294.7905546699921 329.5442325903662


In [6]:
# pg.init()
# pg.display.init()

# window = pg.display.set_mode((WIDTH, HEIGHT))
# clock = pg.time.Clock()
# screen = pg.Surface((WIDTH, HEIGHT))

So every shape inside the pygame is a rect and all the functions of the rect can be used for collide or other stuff for the debug

In [7]:
# just need to take the object inside a variable and use it to need

rect1 = pg.draw.rect(screen, RED, (310, 300, 50, 50))
rect2 = pg.draw.circle(screen, RED, (300, 300), 15)


rect2.colliderect(rect1)

NameError: name 'screen' is not defined

In [65]:
pg.init()
pg.display.init()

window = pg.display.set_mode((WIDTH, HEIGHT))
clock = pg.time.Clock()
screen = pg.Surface((WIDTH, HEIGHT))
screen.fill(WHITE)

last_shot_time = 0
fire_rate = 1000
agent = Agent('asd', 3)
agent.agent_reset(WIDTH, HEIGHT)
agent.current_position = np.array([50, 300], dtype=np.float32)
agent.center = (50, 300)
turret = Turret(WIDTH, HEIGHT)
turret.rotate_turret(agent.center)
# turret.angle = 30

running = True
total_reward = 0
# rays = CastRays(screen)
Walls = {
    'wall1':{'x':500, 'y':100, 'width': 40, 'height': 400},
    # 'wall2':{'x':100, 'y':100, 'width': 40, 'height': 400},
}
while running:
    screen.fill(WHITE)
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
    current_time = pg.time.get_ticks()

    # keys = pg.key.get_pressed()

    # if keys[pg.K_SPACE]:
    if len(turret.get_bullets()) == 0:
        turret.shoot()

    for bullet in turret.get_bullets():
        bullet.move()
        pg.draw.circle(screen, (255, 255, 0), bullet.center, 10)
        turret.auto_destroy()
        # if bullet.pos[0] + 30 > WIDTH or bullet.pos[1] + 30 > HEIGHT:
        #     turret.destroy_bullet(bullet)

        # if bullet.pos[0] - 30 < 0 or bullet.pos[1] - 30 < 0:
        #     turret.destroy_bullet(bullet)
    # bullets = [bullet for bullet in bullets if screen.get_rect().colliderect(bullet.rect)]
    #     # if not screen.get_rect().colliderect(bullet.rect):
    #     #     bullets.remove(bullet)
    #     # else:
    #     #     pg.draw.rect(screen, RED, (bullet.rect.x, bullet.rect.y, bullet.rect.height, bullet.rect.width) )
    

    # pg.draw.rect(screen, GREEN, (turret.rect.x, turret.rect.y, turret.rect.width, turret.rect.height))
    # pg.draw.circle(screen, GREEN, (turret.center), turret.radius)
    pg.draw.circle(screen, GREEN, turret.center, turret.radius)
    pg.draw.line(screen, GREEN, turret.center, turret.rotate_turret(agent.center), 4)

    pg.draw.circle(screen, RED, agent.center, agent.radius)
    # cast_ray = pg.draw.line(screen, GREEN, turret.center, (turret.center[0] + math.cos(math.radians(0))*10000, turret.center[1] +  math.sin(math.radians(0))*10000))


    # wall1 = pg.draw.rect(screen, (0, 0, 255), (500, 100, 40, 400))
    # wall2 = pg.draw.rect(screen, (0, 0, 255), (100, 100, 40, 400))
    # print(wall.x, wall.y, wall.width, wall.height)
    # ray_angles, ray_lengths = update_cast_rays(turret, Walls.values())

    # for ray_length, ray_angle in zip(ray_lengths, ray_angles):
    #     ray_angle = math.radians(ray_angle)
    #     end_point = (int(turret.center[0] + math.cos(ray_angle) * ray_length), int(turret.center[0] + math.sin(ray_angle) * ray_length))
    #     pg.draw.line(screen, GREEN, turret.center, end_point)
    # rays.draw_rays()
    
    # if cast_ray.colliderect(wall):
    #     print(cast_ray.colliderect(wall))

    # for bullet in turret.get_bullets():
        # pg.draw.circle(screen, RED, (bullet.center), bullet.radius)

        

        # if bullet.rect.colliderect(wall1) or bullet.rect.colliderect(wall2):
        #     turret.destroy_bullet(bullet)
        #     total_reward += 1
            # print(total_reward)

    window.blit(screen, screen.get_rect())
    pg.event.pump()
    pg.display.update()
    clock.tick(60)

pg.display.quit()
pg.quit()

In [ ]:
pg.display.quit()
pg.quit()

In [ ]:
angle = 0
start_angle = (angle - 65) # 65 degrees to the left
end_angle = (angle + 65) # 65 degrees to the right

print(start_angle, end_angle)

angle_step = np.abs((start_angle - end_angle)  / 13)  # Adjust the angle step for the number of rays
print(angle_step)
ray_angles = np.arange(start_angle, end_angle + angle_step, angle_step).tolist()
ray_angles = [ray % 360 for ray in ray_angles]
ray_angles = [[math.cos(math.radians(ray)), math.sin(math.radians(ray))] for ray in ray_angles]
ray_angles

-65 65
10.0


[[0.4226182617406996, -0.9063077870366498],
 [0.573576436351046, -0.8191520442889918],
 [0.7071067811865474, -0.7071067811865477],
 [0.8191520442889916, -0.5735764363510465],
 [0.90630778703665, -0.4226182617406992],
 [0.9659258262890683, -0.2588190451025207],
 [0.9961946980917455, -0.08715574274765832],
 [0.9961946980917455, 0.08715574274765817],
 [0.9659258262890683, 0.25881904510252074],
 [0.9063077870366499, 0.42261826174069944],
 [0.8191520442889918, 0.573576436351046],
 [0.7071067811865476, 0.7071067811865476],
 [0.573576436351046, 0.8191520442889918],
 [0.42261826174069944, 0.9063077870366499]]

In [ ]:
angle = 0
start_angle = (angle - 65) # 65 degrees to the left
end_angle = (angle + 65) # 65 degrees to the right

print(start_angle, end_angle)

angle_step = np.abs((start_angle - end_angle)  / 13)  # Adjust the angle step for the number of rays
print(angle_step)
ray_angles = np.arange(start_angle, end_angle + angle_step, angle_step).tolist()
# ray_angles = [ray % 360 for ray in ray_angles]
ray_angles = [[math.cos(math.radians(ray)), math.sin(math.radians(ray))] for ray in ray_angles]
ray_angles

-65 65
10.0


[[0.42261826174069944, -0.9063077870366499],
 [0.573576436351046, -0.8191520442889918],
 [0.7071067811865476, -0.7071067811865476],
 [0.8191520442889918, -0.573576436351046],
 [0.9063077870366499, -0.42261826174069944],
 [0.9659258262890683, -0.25881904510252074],
 [0.9961946980917455, -0.08715574274765817],
 [0.9961946980917455, 0.08715574274765817],
 [0.9659258262890683, 0.25881904510252074],
 [0.9063077870366499, 0.42261826174069944],
 [0.8191520442889918, 0.573576436351046],
 [0.7071067811865476, 0.7071067811865476],
 [0.573576436351046, 0.8191520442889918],
 [0.42261826174069944, 0.9063077870366499]]

In [10]:
import pygame
import math
import numpy as np

from Walls.wall_class import Walls
from Agents.agent import Agent
from Constants.constants import LEVEL_4_WALLS, SCREEN_WIDTH, SCREEN_HEIGHT

# Initialize pygame
pygame.init()
pygame.display.init()
window = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
clock = pygame.time.Clock()
screen = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))

# Create walls
wall = Walls(pygame)
walls = wall.make_wall(LEVEL_4_WALLS)

# Create the agent
agent = Agent("shanila", 15)
agent.agent_reset(SCREEN_WIDTH, SCREEN_HEIGHT)


def cast_and_update_rays(agent, num_rays, walls):
    start_angle = agent.angle - 65  # 65 degrees to the left
    end_angle = agent.angle + 65  # 65 degrees to the right
    angle_step = 130 / (num_rays - 1) if num_rays > 1 else 0  # Adjust the angle step for the number of rays

    ray_angles = np.arange(start_angle, end_angle + angle_step, angle_step).tolist()  # Convert to a Python list
    ray_lengths = []

    for angle in ray_angles:
        ray_direction = np.array([math.cos(math.radians(angle)), math.sin(math.radians(angle))])
        ray_direction /= np.linalg.norm(ray_direction)  # Normalize the direction vector

        length = float('inf')  # Initialize length to infinity

        for wall in walls:
            x1, y1 = agent.current_position
            x2, y2 = agent.current_position + SCREEN_WIDTH * ray_direction  # Set an initial endpoint far away

            x3, y3, x4, y4 = wall['x'], wall['y'], wall['x'] + wall['width'], wall['y'] + wall['height']

            den = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

            if den == 0:
                continue

            t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / den
            u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / den

            if 0 <= t <= 1 and 0 <= u <= 1:
                intersection_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
                wall_length = np.linalg.norm(intersection_point - agent.current_position)
                if wall_length < length:
                    length = wall_length  # Update length if this intersection point is closer

        if length == float('inf'):
            # If no intersections were found, set the length to a finite value (e.g., distance to the screen boundary)
            length = SCREEN_WIDTH  # Adjust this value accordingly

        ray_lengths.append(length)

    return ray_angles, ray_lengths



def render():
    screen.fill((255, 255, 255))
    agent_pos = (int(agent.current_position[0]), int(agent.current_position[1]))
    pygame.draw.circle(screen, (255, 0, 0), agent_pos, agent.radius)
    ray_angles, ray_lengths = cast_and_update_rays(agent, 13, LEVEL_4_WALLS.values())


    for angle, length in zip(ray_angles, ray_lengths):
        if math.isfinite(length):  # Check if the length is finite (ray intersects with a wall)
            end_point = (
                int(agent.current_position[0] + length * math.cos(math.radians(angle))),
                int(agent.current_position[1] + length * math.sin(math.radians(angle)))
            )
            pygame.draw.line(screen, (0, 0, 255), agent_pos, end_point, 2)
        
    for wall in LEVEL_4_WALLS.values():
        pygame.draw.rect(screen, (255, 255, 0), (wall['x'], wall['y'], wall['width'], wall['height']))
    pygame.draw.rect(screen, (0, 0, 0), (650, 0, 150, 150), 2)
    window.blit(screen, screen.get_rect())
    pygame.display.update()
    clock.tick(60)


running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    key = pygame.key.get_pressed()
    if key[pygame.K_UP]:
        agent.step_update(2, SCREEN_WIDTH, SCREEN_HEIGHT)
    if key[pygame.K_LEFT]:
        agent.step_update(1, SCREEN_WIDTH, SCREEN_HEIGHT)
    if key[pygame.K_RIGHT]:
        agent.step_update(0, SCREEN_WIDTH, SCREEN_HEIGHT)

    render()

pygame.display.quit()
pygame.quit()